In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

import sys
from importlib import reload
from sklearn.metrics import confusion_matrix
import tensorflow as tf
#import keras
from scipy import stats
import scipy.signal as scisig
import scipy.stats
import dalmatian
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
#from tqdm.keras import TqdmCallback
from tensorflow.keras.utils import to_categorical

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.layers import LSTM, Dense, Conv1D, TimeDistributed, Flatten, Activation, Dropout, Bidirectional
from keras.callbacks import History, TensorBoard, Callback
import keras.initializers as KI
from keras.layers import BatchNormalization

from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import tensorflow_addons as tfa
from tqdm.keras import TqdmCallback

In [6]:
participants = ["p" + str(i).zfill(2) for i in range(1, 16) if i != 3 and i != 4]
output_path = "../../data/output/E4_comb_mods/"


In [22]:
conf_matrix_list_of_arrays = []
scores = []
acc_per_fold = []
loss_per_fold = []
f1_per_fold = []
prec_per_fold = []
rec_per_fold = []

def s(samples):
    std = np.std(samples)
    if std == 0:
        return samples - np.mean(samples)
    else:
        return (samples - np.mean(samples)) / std

acc_freq = 32
eda_freq = 4
temp_freq = 4
bvp_freq = 64
hr_freq = 1

#sliding window - set window size and step size 
def create_window(pid, win_size=10, win_step=5):
    
    for f in os.listdir(output_path):
        if f.startswith(str(pid)) and f.endswith("ACC.csv"):
            acc_path = f
        elif f.startswith(str(pid)) and f.endswith("EDA.csv"):
            eda_path = f
        elif f.startswith(str(pid)) and f.endswith("TEMP.csv"):
            temp_path = f
        elif f.startswith(str(pid)) and f.endswith("BVP.csv"):
            bvp_path = f
        elif f.startswith(str(pid)) and f.endswith("HR.csv"):
            hr_path = f
    
    acc_data = pd.read_csv(output_path + acc_path, index_col=0 )
    eda_data = pd.read_csv(output_path + eda_path, index_col=0)
    temp_data = pd.read_csv(output_path + temp_path, index_col=0)
    bvp_data = pd.read_csv(output_path + bvp_path, index_col=0)
    hr_data = pd.read_csv(output_path + hr_path, index_col=0)

    bvp_data['interest'] = bvp_data['interest'].replace([1,2,3,4], [0,1,2,3])
    bvp_data['understand'] = bvp_data['understand'].replace([1,2,3,4], [0,1,2,3]) 
    
    acc_x = []; acc_y = []; acc_z = []; eda = []; temp=[]; label=[]; bvp=[]; hr=[]; ut=[]; win_size = 10 ; win_step= 5; 
    total_time = int(len(eda_data) / eda_freq)

    for i in range(win_size, total_time, win_step):

        acc_x.append(s(acc_data["acc_x"][acc_freq * (i - win_size): acc_freq * i]))
        acc_y.append(s(acc_data["acc_y"][acc_freq * (i - win_size): acc_freq * i]))
        acc_z.append(s(acc_data["acc_z"][acc_freq * (i - win_size): acc_freq * i]))

        eda.append(s(eda_data["data"][eda_freq * (i - win_size): eda_freq * i]))
        temp.append(s(temp_data["data"][temp_freq * (i - win_size): temp_freq * i]))
        bvp.append(s(bvp_data["data"][bvp_freq * (i - win_size): bvp_freq * i]))
        #hr.append(s(hr_data["data"][hr_freq * (i - win_size): hr_freq * i]))
        ut.append(s(bvp_data["understand"][bvp_freq * (i - win_size): bvp_freq * i]))

        l = stats.mode(bvp_data["interest"][bvp_freq * (i - win_size): bvp_freq * i])[0][0]
        #l_vote = np.bincount(np.array(l)).argmax()
        label.append(l)


    # return {
    #     "label": np.array(label),
    #     "acc_x": np.array(acc_x), "acc_y": np.array(acc_y), "acc_z": np.array(acc_z),
    #     "eda": np.array(eda), "bvp": np.array(bvp), "temp": np.array(temp), "ut": np.array(ut)}

    return np.array(acc_x), np.array(acc_y), np.array(acc_z),np.array(eda),np.array(bvp),np.array(temp),np.array(ut),np.array(label)

In [37]:
def evaluate(X_train, y_train, X_test, y_test):

    n_timesteps = X_train[0].shape[1]
    n_features = X_train[0].shape[2]

    input_shape = (n_timesteps,n_features)
    
    model = Sequential()
    model.add(Conv1D(filters=128,kernel_size=5,activation='relu',input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=4))

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
    #model.add(MaxPooling1D(pool_size=4))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

    X_tr = np.vstack([X_train[i] for i in range(len(X_train))])
    y_tr = np.vstack([y_train[i] for i in range(len(y_train))])

    X_te = np.vstack([X_test[i] for i in range(len(X_test))])
    y_te = np.vstack([y_test[i] for i in range(len(y_test))])
    
    y_tr = to_categorical(y_tr)
    y_te = to_categorical(y_te)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
    #class_weights = class_weight.compute_class_weight('balanced',np.unique(y_tr),y_tr)
    
    history = model.fit(X_tr, y_tr, epochs=5, validation_data=(X_te,y_te),batch_size=32, verbose=1, callbacks=[es])
# evaluate model
    _, accuracy = model.evaluate(X_te, y_te, batch_size=32, verbose=1)
    print ('Test accuracy :%.2f' % (accuracy))
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

In [23]:
comb_data = [create_window(x) for x in participants]

In [39]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=49)
# K-fold Cross Validation model evaluation
fold_no = 1

X_train_data = []; y_train_data=[]; X_test_data=[]; y_test_data=[]

for train, test in kfold.split(comb_data):

    
    # # X_train_data.append(comb_data[train][0])
    # # y_train_data.append(comb_data[train][1])

    # # X_test_data.append(comb_data[test][0])
    # # y_test_data.append(comb_data[test][1])
    X_train_data = [comb_data[i][:7] for i in train]
    y_train_data = [comb_data[i][7:] for i in train]

    X_test_data = [comb_data[i][:7] for i in test]
    y_test_data = [comb_data[i][7:] for i in test]

    print (X_train_data)
    
    #evaluate(X_train_data, y_train_data, X_test_data, y_test_data)

[(array([[-0.1382327 , -0.1382327 , -0.1382327 , ..., -0.1382327 ,
        -0.1382327 , -0.1382327 ],
       [-0.04070815, -0.04070815, -0.04070815, ..., -4.38291069,
        -4.38291069, -0.04070815],
       [ 0.17519545,  0.17519545,  3.28978116, ...,  0.17519545,
         0.17519545, -2.93939027],
       ...,
       [ 0.48069238,  0.48069238,  0.48069238, ...,  0.48069238,
         0.48069238,  0.48069238],
       [ 0.16415595, -2.1197529 , -2.1197529 , ...,  0.16415595,
         0.16415595,  0.16415595],
       [-0.35659143, -0.35659143, -0.35659143, ...,  2.64628379,
        -0.35659143, -0.35659143]]), array([[-0.13018891, -0.13018891, -0.13018891, ...,  5.07736753,
        -0.13018891, -0.13018891],
       [-0.22176638, -0.22176638, -0.22176638, ..., -0.22176638,
        -0.22176638, -0.22176638],
       [-0.1382327 , -0.1382327 , -0.1382327 , ..., -0.1382327 ,
        -0.1382327 , -0.1382327 ],
       ...,
       [-0.43033148, -0.43033148, -0.43033148, ...,  2.32379001,
       

320